위키독스 '딥 러닝을 이용한 자연어 처리 입문'의 한국어 BERT를 이용한 개체명 인식 튜토리얼.

In [ ]:
pip install transformers

     |████████████████████████████████| 3.3 MB 5.5 MB/s 
     |████████████████████████████████| 895 kB 42.3 MB/s 
     |████████████████████████████████| 3.3 MB 41.2 MB/s 
     |████████████████████████████████| 596 kB 46.6 MB/s 
     |████████████████████████████████| 61 kB 448 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
pip install seqeval

     |████████████████████████████████| 43 kB 906 kB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=3fd0b55fba9dcefeb5a8e6240c2f7083a2e3f44452ee87bdd167ea207d0e95f4
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import f1_score, classification_report
import tensorflow as tf

# 1. 데이터 로드

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1jatBP8yZkWn6Kg6mjN7nWLnYVwXE_sY_' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1jatBP8yZkWn6Kg6mjN7nWLnYVwXE_sY_" -O ner_train_data.csv && rm -rf /tmp/cookies.txt

--2021-12-13 07:31:37--  https://docs.google.com/uc?export=download&confirm=&id=1jatBP8yZkWn6Kg6mjN7nWLnYVwXE_sY_
Resolving docs.google.com (docs.google.com)... 142.250.159.139, 142.250.159.102, 142.250.159.138, ...
Connecting to docs.google.com (docs.google.com)|142.250.159.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-1g-docs.googleusercontent.com/docs/securesc/m71qcjl1jljhs2ocfkpjqhfbhhjcb0u4/mclcv2f8io78571ba80s75qt6vle08ki/1639380675000/17609157229046208934/11939825763431874016Z/1jatBP8yZkWn6Kg6mjN7nWLnYVwXE_sY_?e=download [following]
--2021-12-13 07:31:38--  https://doc-0o-1g-docs.googleusercontent.com/docs/securesc/m71qcjl1jljhs2ocfkpjqhfbhhjcb0u4/mclcv2f8io78571ba80s75qt6vle08ki/1639380675000/17609157229046208934/11939825763431874016Z/1jatBP8yZkWn6Kg6mjN7nWLnYVwXE_sY_?e=download
Resolving doc-0o-1g-docs.googleusercontent.com (doc-0o-1g-docs.googleusercontent.com)... 173.194.197.132, 2607:f8b0:4001:c1b::84
Connectin

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YVYShKCtWfigXBOb5ie7s6QmA-dHnjt3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1YVYShKCtWfigXBOb5ie7s6QmA-dHnjt3" -O ner_test_data.csv && rm -rf /tmp/cookies.txt

--2021-12-13 07:31:39--  https://docs.google.com/uc?export=download&confirm=&id=1YVYShKCtWfigXBOb5ie7s6QmA-dHnjt3
Resolving docs.google.com (docs.google.com)... 142.250.159.113, 142.250.159.139, 142.250.159.102, ...
Connecting to docs.google.com (docs.google.com)|142.250.159.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-14-docs.googleusercontent.com/docs/securesc/pel47mcss5878mf4bs7hjatusollj2r7/ga0ulo3utadj09l8hk0lu2tok3i5sb4i/1639380675000/17609157229046208934/10535431281839400414Z/1YVYShKCtWfigXBOb5ie7s6QmA-dHnjt3?e=download [following]
--2021-12-13 07:31:41--  https://doc-0g-14-docs.googleusercontent.com/docs/securesc/pel47mcss5878mf4bs7hjatusollj2r7/ga0ulo3utadj09l8hk0lu2tok3i5sb4i/1639380675000/17609157229046208934/10535431281839400414Z/1YVYShKCtWfigXBOb5ie7s6QmA-dHnjt3?e=download
Resolving doc-0g-14-docs.googleusercontent.com (doc-0g-14-docs.googleusercontent.com)... 173.194.197.132, 2607:f8b0:4001:c1b::84
Connectin

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1_DPfdY1Q5Xt2md7QVbKcQLRDYHm3qgVQ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1_DPfdY1Q5Xt2md7QVbKcQLRDYHm3qgVQ" -O ner_label.txt && rm -rf /tmp/cookies.txt

--2021-12-13 07:31:42--  https://docs.google.com/uc?export=download&confirm=&id=1_DPfdY1Q5Xt2md7QVbKcQLRDYHm3qgVQ
Resolving docs.google.com (docs.google.com)... 142.250.159.113, 142.250.159.139, 142.250.159.102, ...
Connecting to docs.google.com (docs.google.com)|142.250.159.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-2c-docs.googleusercontent.com/docs/securesc/v5o160kjjvqfbho7ufnmvo76401be2bs/3edk16rj46andq6569jss06qho53vfts/1639380675000/17609157229046208934/18187602189213074178Z/1_DPfdY1Q5Xt2md7QVbKcQLRDYHm3qgVQ?e=download [following]
--2021-12-13 07:31:42--  https://doc-14-2c-docs.googleusercontent.com/docs/securesc/v5o160kjjvqfbho7ufnmvo76401be2bs/3edk16rj46andq6569jss06qho53vfts/1639380675000/17609157229046208934/18187602189213074178Z/1_DPfdY1Q5Xt2md7QVbKcQLRDYHm3qgVQ?e=download
Resolving doc-14-2c-docs.googleusercontent.com (doc-14-2c-docs.googleusercontent.com)... 173.194.197.132, 2607:f8b0:4001:c1b::84
Connectin

In [ ]:
train_ner_df = pd.read_csv("ner_train_data.csv")

In [ ]:
train_ner_df.head()

,Sentence,Tag
0,"정은 씨를 힘들게 한 가스나그, 가만둘 수 없겠죠 .",PER-B O O O O O O O O
1,▶ 쿠마리 한동수가 말하는 '가넷 & 에르덴',O PER-B PER-I O PER-B O PER-B
2,슈나이더의 프레젠테이션은 말 청중을 위한 특별한 쇼다 .,PER-B O O CVL-B O O O O
3,지구 최대 연료탱크 수검 회사 구글이 연내 22명 안팎의 인력을 갖춘 연구개발(R&...,O O TRM-B O O ORG-B DAT-B NUM-B O O O ORG-B LO...
4,5. <10:00:TI_HOUR> 도이치증권대 <0:1:QT_SPORTS> 연예오락...,NUM-B O ORG-B O ORG-B


SK하이닉스가 상장을 했다.  

SK, 하이닉스, 가, 상장, 을, 했다, .  

b-stock, i-stock, o, o, o, o, o

sk, 하이, 닉스, 가, 상, 장, 을, 했, 다, .

b-stock, i-stock, i-stock, o, o, o, o, o, o, o

In [ ]:
test_ner_df = pd.read_csv("ner_test_data.csv")

In [ ]:
test_ner_df.head()

,Sentence,Tag
0,"라티은-원윤정, 휘닉스파크클래식 프로골퍼",PER-B EVT-B CVL-B
1,5원으로 맺어진 애인까지 돈이라는 민감한 원자재를 통해 현대인의 물질만능주의를 꼬집...,NUM-B O O O O O O O O O O O FLD-B O
2,-날로 삼키면 맛이 어떤지 일차 드셔보시겠어요 .,O O O O NUM-B O O
3,"-네, 지었습니다 .",O O O
4,◇신규 투자촉진에 방점=이번 접속료 조정결과에서 눈에 띄는 지점은 WCDMA/HSD...,O O O O O O O O TRM-B O TRM-B TRM-I ORG-B O TR...


훈련 데이터와 테스트 데이터의 샘플 개수를 확인해봅시다.

In [ ]:
print("학습 데이터 샘플 개수 :", len(train_ner_df))
print("테스트 데이터 샘플 개수 :", len(test_ner_df))

학습 데이터 샘플 개수 : 81000
테스트 데이터 샘플 개수 : 9000


훈련 데이터와 테스트 데이터에서 문장과 레이블을 각각 분리하여 저장.

In [ ]:
train_data_sentence = [sent.split() for sent in train_ner_df['Sentence'].values]
test_data_sentence = [sent.split() for sent in test_ner_df['Sentence'].values]
train_data_label = [tag.split() for tag in train_ner_df['Tag'].values]
test_data_label = [tag.split() for tag in test_ner_df['Tag'].values]

임의로 훈련 데이터에서 2번 인덱스의 문장과 레이블을 출력.

In [ ]:
print(train_data_sentence[2])
print(train_data_label[2])

['슈나이더의', '프레젠테이션은', '말', '청중을', '위한', '특별한', '쇼다', '.']
['PER-B', 'O', 'O', 'CVL-B', 'O', 'O', 'O', 'O']


이 데이터는 형태소 단위가 아니라 어절 단위(띄어쓰기 단위)로 개체명 인식 레이블이 태깅되었다는 특징이 있음. ner_label.txt 파일을 읽어서 개체명 태깅 정보의 종류를 확인.

In [ ]:
labels = [label.strip() for label in open('ner_label.txt', 'r', encoding='utf-8')]
print('개체명 태깅 정보 :', labels)

개체명 태깅 정보 : ['O', 'PER-B', 'PER-I', 'FLD-B', 'FLD-I', 'AFW-B', 'AFW-I', 'ORG-B', 'ORG-I', 'LOC-B', 'LOC-I', 'CVL-B', 'CVL-I', 'DAT-B', 'DAT-I', 'TIM-B', 'TIM-I', 'NUM-B', 'NUM-I', 'EVT-B', 'EVT-I', 'ANM-B', 'ANM-I', 'PLT-B', 'PLT-I', 'MAT-B', 'MAT-I', 'TRM-B', 'TRM-I']


레이블을 읽어서 각 개체명 태깅 정보와 정수를 맵핑하는 딕셔너리를 만듬. tag_to_index는 개체명 태깅 정보를 key, 정수를 value로 하는 딕셔너리이며 index_to_tag는 그 반대 형태를 가지는 딕셔너리.

In [ ]:
tag_to_index = {tag: index for index, tag in enumerate(labels)}
index_to_tag = {index: tag for index, tag in enumerate(labels)}

In [ ]:
print(tag_to_index)
print(index_to_tag)

{'O': 0, 'PER-B': 1, 'PER-I': 2, 'FLD-B': 3, 'FLD-I': 4, 'AFW-B': 5, 'AFW-I': 6, 'ORG-B': 7, 'ORG-I': 8, 'LOC-B': 9, 'LOC-I': 10, 'CVL-B': 11, 'CVL-I': 12, 'DAT-B': 13, 'DAT-I': 14, 'TIM-B': 15, 'TIM-I': 16, 'NUM-B': 17, 'NUM-I': 18, 'EVT-B': 19, 'EVT-I': 20, 'ANM-B': 21, 'ANM-I': 22, 'PLT-B': 23, 'PLT-I': 24, 'MAT-B': 25, 'MAT-I': 26, 'TRM-B': 27, 'TRM-I': 28}
{0: 'O', 1: 'PER-B', 2: 'PER-I', 3: 'FLD-B', 4: 'FLD-I', 5: 'AFW-B', 6: 'AFW-I', 7: 'ORG-B', 8: 'ORG-I', 9: 'LOC-B', 10: 'LOC-I', 11: 'CVL-B', 12: 'CVL-I', 13: 'DAT-B', 14: 'DAT-I', 15: 'TIM-B', 16: 'TIM-I', 17: 'NUM-B', 18: 'NUM-I', 19: 'EVT-B', 20: 'EVT-I', 21: 'ANM-B', 22: 'ANM-I', 23: 'PLT-B', 24: 'PLT-I', 25: 'MAT-B', 26: 'MAT-I', 27: 'TRM-B', 28: 'TRM-I'}


개체명 태깅 정보의 총 개수를 출력해봅시다.

In [ ]:
tag_size = len(tag_to_index)
print('개체명 태깅 정보의 개수 :',tag_size)

개체명 태깅 정보의 개수 : 29


# 2. 전처리 예시

In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

전처리 과정을 이해. 임의로 훈련 데이터 중 1번 인덱스의 문장과 레이블을 선택하고 이에 대해서 전처리를 진행. 해당 문장과, 레이블, 그리고 레이블에 대해서 정수 인코딩을 진행한 결과는 다음과 같음.

In [ ]:
sent = train_data_sentence[1]
label = train_data_label[1]

In [ ]:
print('문장 :', sent)
print('레이블 :',label)
print('레이블의 정수 인코딩 :',[tag_to_index[idx] for idx in label])
print('문장의 길이 :', len(sent))
print('레이블의 길이 :', len(label))

문장 : ['▶', '쿠마리', '한동수가', '말하는', "'가넷", '&', "에르덴'"]
레이블 : ['O', 'PER-B', 'PER-I', 'O', 'PER-B', 'O', 'PER-B']
레이블의 정수 인코딩 : [0, 1, 2, 0, 1, 0, 1]
문장의 길이 : 7
레이블의 길이 : 7


위 문장과 레이블을 예시로 들어 BERT 토크나이저를 적용하는 과정을 이해. 이미 위와 같이 문장에 단어 토큰화가 진행되어져 있는 상황에서 BERT 토크나이저를 적용하기 위해서는 각 단어에 대해서 BERT 토크나이저를 사용하여 단어를 서브워드로 분리. 위 문장에 BERT 토크나이저를 적용한 결과는 다음과 같음.


In [ ]:
tokens = []

for one_word in sent:
  # 각 단어에 대해서 서브워드로 분리.
  # ex) one_word = '쿠마리' ===> subword_tokens = ['쿠', '##마리']
  # ex) one_word = '한동수가' ===> subword_tokens = ['한동', '##수', '##가']
  subword_tokens = tokenizer.tokenize(one_word)
  tokens.extend(subword_tokens)

print('BERT 토크나이저 적용 후 문장 :',tokens)
print('레이블 :', label)
print('레이블의 정수 인코딩 :',[tag_to_index[idx] for idx in label])
print('문장의 길이 :', len(tokens))
print('레이블의 길이 :', len(label))

BERT 토크나이저 적용 후 문장 : ['▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'"]
레이블 : ['O', 'PER-B', 'PER-I', 'O', 'PER-B', 'O', 'PER-B']
레이블의 정수 인코딩 : [0, 1, 2, 0, 1, 0, 1]
문장의 길이 : 16
레이블의 길이 : 7


'쿠마리'가 '쿠'와 '##마리'로 분리되는 등 단어들이 서브워드로 분리됨. 문제는 이렇게 되면 문장의 길이와 레이블의 길이가 달라지게 됨. 레이블의 길이도 문장의 길이와 일치하도록 추가적인 처리를 진행해야 함. '쿠마리'의 레이블은 'PER-B'. 그렇다면 '쿠'와 '##마리'의 레이블은 어떻게 해야할까?  

이 경우 첫번째 서브워드에 대해서만 기존의 레이블을 부여하고 뒤에 생겨난 서브워드들에 대해서는 레이블을 주지 않는 방법이 있음. '쿠마리'가 'PER-B'의 레이블을 가지고 있었다면, 분리된 '쿠', '##마리'에 대해서 '쿠'에는 'PER-B'를 부여하고 그 뒤의 서브워드인 '##마리'에 대해서는 레이블을 주지 않는 것. 그 방법으로는 레이블을 정수 인코딩하는 과정에서 첫번째 서브워드가 아닌 경우에는 -100을 부여하는 방식을 사용.

In [ ]:
tokens = []
labels_ids = []

for one_word, label_token in zip(train_data_sentence[1], train_data_label[1]):
  subword_tokens = tokenizer.tokenize(one_word)
  tokens.extend(subword_tokens)
  labels_ids.extend([tag_to_index[label_token]]+ [-100] * (len(subword_tokens) - 1))

print('토큰화 후 문장 :',tokens)
print('레이블 :', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in labels_ids])
print('레이블의 정수 인코딩 :', labels_ids)
print('문장의 길이 :', len(tokens))
print('레이블의 길이 :', len(labels_ids))

토큰화 후 문장 : ['▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'"]
레이블 : ['O', 'PER-B', '[PAD]', 'PER-I', '[PAD]', '[PAD]', 'O', '[PAD]', '[PAD]', 'PER-B', '[PAD]', '[PAD]', 'O', 'PER-B', '[PAD]', '[PAD]']
레이블의 정수 인코딩 : [0, 1, -100, 2, -100, -100, 0, -100, -100, 1, -100, -100, 0, 1, -100, -100]
문장의 길이 : 16
레이블의 길이 : 16


레이블의 정수 인코딩 결과를 보면 '쿠'에 대해서는 PER-B에 해당하는 정수 1을 부여하였지만, #마리에 대해서는 -100을 부여. 마찬가지로 기존에는 '한동수가'에 PER-I가 부여되어 있었으나, 서브워드 토큰화 과정에서 ['한동', '##수', '##가']로 분리되었고, 첫번째 서브워드인 '한동'에는 PER-I에 해당하는 정수인 2를 부여하지만 그 뒤의 서브워드들인 '##수', '##가'에 대해서는 -100을 부여. '말하는'은 레이블이 'O'였으나 ['말', '##'하, '##는']으로 분리되었고 '말'에 대해서는 'O'에 해당하는 정수인 0을 부여하지만, 그 뒤의 서브워드 들에 대해서는 -100을 부여.  

-100을 부여하고나서는 실질적으로 학습할 때는 해당 레이블에 대해서는 학습을 무시하는 정책을 취할 예정. 정확히는 -100을 레이블에 대한 패딩 토큰 [PAD]로 사용. 레이블에 대해서는 문장의 길이를 맞추기 위해서도 -100을 사용. 그리고 이후 손실 함수에서 -100을 무시하도록 추가 처리를 진행.  

레이블에 대한 전처리를 이해하였다면, 이제 문장과 레이블에 대한 정수 인코딩. 그리고 세그먼트 인코딩과 어텐션 마스크까지 생성하는 함수 convert_examples_to_features를 구현.

# 3. 전처리

In [ ]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        tokens = []
        labels_ids = []
        for one_word, label_token in zip(example, label):
            # 하나의 단어에 대해서 서브워드로 토큰화
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            # 서브워드 중 첫번째 서브워드만 개체명 레이블을 부여하고 그 외에는 -100으로 채운다.
            labels_ids.extend([tag_to_index[label_token]]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        # [CLS]와 [SEP]를 후에 추가할 것을 고려하여 최대 길이를 초과하는 샘플의 경우 max_seq_len - 2의 길이로 변환.
        # ex) max_seq_len = 64라면 길이가 62보다 긴 샘플은 뒷 부분을 자르고 길이 62로 변환.
        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]

        # [SEP]를 추가하는 코드
        # 1. 토큰화 결과의 맨 뒷 부분에 [SEP] 토큰 추가
        # 2. 레이블에도 맨 뒷 부분에 -100 추가.
        tokens += [sep_token]
        labels_ids += [pad_token_id_for_label]

        # [CLS]를 추가하는 코드
        # 1. 토큰화 결과의 앞 부분에 [CLS] 토큰 추가
        # 2. 레이블의 맨 앞 부분에도 -100 추가.
        tokens = [cls_token] + tokens
        labels_ids = [pad_token_id_for_label] + labels_ids

        # 정수 인코딩
        input_id = tokenizer.convert_tokens_to_ids(tokens)

        # 어텐션 마스크 생성
        attention_mask = [1] * len(input_id)

        # 정수 인코딩에 추가할 패딩 길이 연산
        padding_count = max_seq_len - len(input_id)

        # 정수 인코딩, 어텐션 마스크에 패딩 추가
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)

        # 세그먼트 인코딩.
        token_type_id = [pad_token_id_for_segment] * max_seq_len


        # 레이블 패딩. (단, 이 경우는 패딩 토큰의 ID가 -100)
        label = labels_ids + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label) == max_seq_len, "Error with labels length {} vs {}".format(len(label), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

훈련 데이터와 테스트 데이터에 대해서 전처리를 진행하고 결과를 살펴봄. 단, 문장의 최대 길이는 임의로 128로 지정.

In [ ]:
X_train, y_train = convert_examples_to_features(train_data_sentence, train_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 81000/81000 [00:59<00:00, 1355.95it/s]


In [ ]:
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 9000/9000 [00:06<00:00, 1370.42it/s]


훈련 데이터의 첫번째 샘플에 대해서 기존에 주어졌던 원문과 레이블이 BERT 토크나이저 적용 후 어떻게 변경되었는지 보고, 그 후에 실질적으로 BERT의 입력이 되는 정수 인코딩 결과는 어떻게 되었는지 살펴봄.

In [ ]:
print('기존 원문 :', train_data_sentence[0])
print('기존 레이블 :', train_data_label[0])
print('-' * 50)
print('토큰화 후 원문 :', [tokenizer.decode([word]) for word in X_train[0][0]])
print('토큰화 후 레이블 :', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in y_train[0]])
print('-' * 50)
print('정수 인코딩 결과 :', X_train[0][0])
print('정수 인코딩 레이블 :', y_train[0])

기존 원문 : ['정은', '씨를', '힘들게', '한', '가스나그,', '가만둘', '수', '없겠죠', '.']
기존 레이블 : ['PER-B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
--------------------------------------------------
토큰화 후 원문 : ['[CLS]', '정은', '씨', '##를', '힘들', '##게', '한', '가스', '##나', '##그', ',', '가만', '##둘', '수', '없', '##겠', '##죠', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'

'토큰화 후 레이블'과 '정수 인코딩 레이블' 결과를 비교해서 보면, 첫번째 서브워드에는 기존의 레이블을 부여하고, 그 뒤의 서브워드들에 대해서는 -100을 부여. 그 외에는 [CLS] 토큰과 [SEP] 토큰의 위치에 대해서도 별도 개체명 예측이 의미는 없으므로 -100을 부여.

In [ ]:
print('-' * 50)
print('세그먼트 인코딩 :', X_train[2][0])
print('어텐션 마스크 :', X_train[1][0])

--------------------------------------------------
세그먼트 인코딩 : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


세그먼트 인코딩과 어텐션 마스크의 경우 네이버 영화 리뷰 분류와 같은 이전에 풀었던 문제들과 부여하는 방식이 다르지 않은데, 세그먼트 인코딩에서는 개체명 인식은 두 개 이상의 문장을 구분할 필요가 없는 문제이므로 전부 0으로 채워줌. 어텐션 마스크의 경우에는 [CLS]부터 [SEP]까지는 1을 부여하고, [SEP] 이후의 [PAD]에 대해서는 0을 부여.

훈련 데이터의 두번째 샘플에 대해서 출력.

In [ ]:
print('기존 원문 :', train_data_sentence[1])
print('기존 레이블 :', train_data_label[1])
print('-' * 50)
print('토큰화 후 원문 :', [tokenizer.decode([word]) for word in X_train[0][1]])
print('토큰화 후 레이블 :', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in y_train[1]])
print('-' * 50)
print('정수 인코딩 결과 :', X_train[0][1])
print('정수 인코딩 레이블 :', y_train[1])
print('-' * 50)
print('세그먼트 인코딩 :', X_train[2][1])
print('어텐션 마스크 :', X_train[1][1])

기존 원문 : ['▶', '쿠마리', '한동수가', '말하는', "'가넷", '&', "에르덴'"]
기존 레이블 : ['O', 'PER-B', 'PER-I', 'O', 'PER-B', 'O', 'PER-B']
--------------------------------------------------
토큰화 후 원문 : ['[CLS]', '▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'", '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[P

이제 개체명 인식을 위한 모델을 설계해봅시다. 텍스트 분류를 수행할 때는 BERT의 출력인 outputs에서 인덱스 1로 접근. outputs[1]은 [CLS] 토큰에 접근하는 방법으로 Many-to-One 문제에 해당하는 텍스트 분류를 풀 때 사용. 반면, 이번에는 모든 입력에 대해서 출력을 수행해야 하는 Many-to-Many 문제에 해당하므로 outputs[0]을 사용하여 문제를 풀어야 함.

outputs[0]과 연결되는 출력층에는 레이블의 개수인 num_labels를 전달. 이번에는 다중 클래스 분류 문제임에도 출력층에 소프트맥스 함수를 사용하지 않았는데, 출력층에 소프트맥스 함수를 사용하지 않았다면 손실 함수에서 이를 고려하도록 구현하면 됨. 이에 대해서는 손실 함수를 설명할 때 재언급.

# 4. 모델링

In [ ]:
class TFBertForTokenClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFBertForTokenClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        # 전체 시퀀스에 대해서 분류해야 하므로 outputs[0]임에 주의
        all_output = outputs[0]
        prediction = self.classifier(all_output)

        return prediction

# 5. 손실 함수에서 -100 레이블은 제외시키기

앞서 레이블이 -100인 경우에 대해서는 손실 함수에서 오차를 구현하지 않도록 할 예정이라고 언급. 다시 말해서 레이블이 -100인 경우에 대해서는 학습에 반영하지 않고자 함. 이를 어떻게 구현할 수 있는지 임의의 다중 클래스 분류 모델을 가정하고 살펴봄.  

labels를 실제값에 해당하는 레이블. logits을 어떤 모델의 예측값이라고 가정. 우리가 지금가지 다뤄왔던 다중 클래스 분류 모델은 처음부터 정수 레이블을 예측하지 않음. 기본적으로 각 레이블이 정답일 확률값을 예측. 가정하기를 현재 가정하고 있는 모델이 예측해야 하는 레이블의 개수가 3개라고 해보기. 다시 말해 logits의 각 예측값은 3차원 벡터여야만 함.

In [ ]:
labels = tf.constant([[-100, 2, 1, -100]])
logits = tf.constant([[[0.8, 0.1, 0.1], [0.06, 0.04, 0.9], [0.75, 0.1, 0.15], [0.4, 0.5, 0.1]]])

In [ ]:
active_loss = tf.reshape(labels, (-1,)) != -100
print(active_loss)

tf.Tensor([False  True  True False], shape=(4,), dtype=bool)


In [ ]:
reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
print(reduced_logits)

tf.Tensor(
[[0.06 0.04 0.9 ]
 [0.75 0.1  0.15]], shape=(2, 3), dtype=float32)


In [ ]:
labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
print(labels)

tf.Tensor([2 1], shape=(2,), dtype=int32)


In [ ]:
tf.keras.losses.SparseCategoricalCrossentropy()(labels, reduced_logits)

<tf.Tensor: shape=(), dtype=float32, numpy=1.2039728>

In [ ]:
def compute_loss(labels, logits):

  # 다중 클래스 분류 문제에서 소프트맥스 함수 미사용 시 from_logits=True로 설정.
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=tf.keras.losses.Reduction.NONE)

  # activat_loss는 -100인 경우에는 False, 아닌 경우에는 True를 리턴
  # ex) labels = [-100, 2, 1, -100] ===> active_loss = [False True True False]
  active_loss = tf.reshape(labels, (-1,)) != -100

  # loss_fun으로 넣기 전 active_loss를 참고하여
  # labels와 logits에 대해서 -100의 위치에 대해서는 값을 제외
  reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
  labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)

  return loss_fn(labels, reduced_logits)

레이블 : [1 -100 2 -100]

모델의 예측 : [1 5 3 5]

-100인 위치는 제거 후 무시하고 loss를 계산하도록 loss function 수정.  

레이블 : [1 2]  
모델의 예측 : [1 3]

# 6. 학습

In [ ]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.102.43.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.102.43.122:8470


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


손실 함수로는 다중 클래스 분류 문제이므로 우리가 지금까지 다중 클래스 분류 문제에서 사용해왔던 것처럼 SparseCategoricalCrossentropy를 사용. 다만, 우리는 -100이라는 숫자를 레이블에 대한 [PAD] 토큰으로 간주하여 loss를 구하지 않는 것으로 가정하고 전처리를 진행하였으므로 손실 함수에서도 이를 반영해주어야 함.

한 가지 더 고려해야할 점은 앞서 만든 TFBertForTokenClassification 모델에서 출력층에 소프트맥스 함수를 사용하지 않아 예측 벡터의 총 합이 1이 되지 않은 상태라는 점. 이 경우 SparseCategoricalCrossentropy에서 from_logits를 True로 지정하면 예측값이 소프트맥스 함수를 통과하지 않았음을 고려하고 오차를 구함.

이에 대한 설명은 아래의 SparseCategoricalCrossentropy의 공식 문서에서 확인할 수 있음.

https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy

공식 문서에 따르면 from_logits의 값을 지정해주지 않으면 기본값은 False로 SparseCategoricalCrossentropy는 기본적으로 예측값이 소프트맥스 함수값을 통과한 상태임을 가정. 하지만 예측값에 소프트맥스 함수를 통과시키지 않은 경우라면, from_logits=True를 해주면 이를 감안하여 오차를 구하게 됨.

In [ ]:
with strategy.scope():
  model = TFBertForTokenClassification("klue/bert-base", num_labels=tag_size)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  model.compile(optimizer=optimizer, loss=compute_loss)

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'bert.embeddings.position_ids', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

한 에포크가 끝날 때마다 테스트 데이터에 대해서 F1 score를 계산하여 학습이 과적합되고 있지는 않은지 판단하고자 함. 이를 위해서 tf.keras.callbacks.Callback를 상속받아서 커스텀 콜백 클래스인 F1score를 구현.

학습 중 모델의 동작을 정의하는 커스텀 콜백을 구현하는 방식에 대해서는 아래의 링크를 참고하여 구현.  

커스텀 콜백 구현 방법 : https://www.tensorflow.org/guide/keras/custom_callback?hl=ko  

커스텀 콜백 클래스에 on_epoch_end라는 함수를 정의하면, 에포크가 끝날 때마다 해당 함수가 호출됨.

In [ ]:
class F1score(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test

    def sequences_to_tags(self, label_ids, pred_ids):
      label_list = []
      pred_list = []

      for i in range(0, len(label_ids)):
        label_tag = []
        pred_tag = []

        # 레이블의 값이 -100인 경우는 F1 score 계산 시에도 제외
        # ex) 레이블 디코딩 과정
        # label_index : [1 -100 2 -100] ===> [1 2] ===> label_tag : [PER-B PER-I]
        for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
          if label_index != -100:
            label_tag.append(index_to_tag[label_index])
            pred_tag.append(index_to_tag[pred_index])
        
        label_list.append(label_tag)
        pred_list.append(pred_tag)

      return label_list, pred_list

    # 에포크가 끝날 때마다 실행되는 함수}
    def on_epoch_end(self, epoch, logs={}):

      y_predicted = self.model.predict(self.X_test)
      y_predicted = np.argmax(y_predicted, axis = 2)

      label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)

      score = f1_score(label_list, pred_list, suffix=True)
      print(' - f1: {:04.2f}'.format(score * 100))
      print(classification_report(label_list, pred_list, suffix=True))

f1 스코어를 계산하기 위해서는 예측값과 레이블이 정수 시퀀스가 아니라 개체명 태깅 정보들의 시퀀스이어야만 함. 정수 시퀀스로부터 개체명 태깅 정보 시퀀스로 변환해주는 함수가 sequences_to_tags라는 함수.  

해당 함수의 동작 방식을 살펴보기. 예를 들어 실제값에 해당하는 레이블(y_test)이 [-100 1 -100 2 -100 -100]이고 예측값(y_predicted)이 [0 1 0 2 0 0]이라고 해보자. 우리는 여기서 레이블의 값이 -100인 경우에 대해서는 고려하지 않으며 레이블의 값이 -100인 위치에 대해서 레이블과 예측값 모두 제외. 이에 따라 레이블과 예측값은 각각 [1 2], [1 2]가 됨. 이를 개체명 태깅 정보 시퀀스로 변환하면 실제값과 예측값은 [PER-B, PER-I]와 [PER-B, PER-I]가 됨. 그리고 이 결과를 f1 score 계산에 반영. 이 경우 예측값이 실제값을 모두 정확하게 맞춘 경우가 됨.

In [ ]:
f1_score_report = F1score(X_test, y_test)

In [ ]:
model.fit(
    X_train, y_train, epochs=3, batch_size=32,
    callbacks = [f1_score_report]
)

Epoch 1/3


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None, 128) dtype=int32>]


Instructions for updating:
use `experimental_local_results` instead.


Instructions for updating:
use `experimental_local_results` instead.
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None, 128) dtype=int32>]


   6/2532 [..............................] - ETA: 3:16 - loss: 2.2431WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0067s vs `on_train_batch_end` time: 7.8684s). Check your callbacks.


2532/2532 [==============================] - ETA: 0s - loss: 0.2726

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 128) dtype=int64>]


 - f1: 84.91
              precision    recall  f1-score   support

         AFW       0.63      0.53      0.58       394
         ANM       0.71      0.76      0.73       701
         CVL       0.80      0.85      0.82      5758
         DAT       0.91      0.93      0.92      2521
         EVT       0.73      0.77      0.75      1094
         FLD       0.76      0.46      0.57       228
         LOC       0.82      0.86      0.84      2126
         MAT       0.22      0.17      0.19        12
         NUM       0.90      0.93      0.91      5590
         ORG       0.85      0.87      0.86      4086
         PER       0.89      0.89      0.89      4426
         PLT       0.42      0.15      0.22        34
         TIM       0.81      0.91      0.86       314
         TRM       0.74      0.73      0.74      1964

   micro avg       0.84      0.86      0.85     29248
   macro avg       0.73      0.70      0.71     29248
weighted avg       0.84      0.86      0.85     29248

2532/2532 [=

이제 학습하지 않은 임의의 문장에 대해서 개체명 태깅 정보를 예측. 임의의 문장에 대한 예측을 위해 전처리를 수행하는 함수인 convert_examples_to_features_for_prediction를 구현. 기본적으로 해당 함수가 input_ids, attention_masks, token_type_ids를 준비하는 방식은 학습 단계에서 전처리를 위해 사용했던 함수인 convert_examples_to_features와 같음. 학습 단계나 테스트 단계나 BERT의 입력 형식은 동일하기 때문.

예측을 위한 함수인 convert_examples_to_features_for_prediction를 구현 시 고려해야할 점은 학습 단계에서는 레이블이 있었으나 임의의 문장에 대한 예측 시에는 레이블이 존재하지 않는다는 점. 앞서 f1 score를 계산할 때는 -100의 위치를 레이블인 y_test로 파악. 하지만 이번에는 레이블이 존재하지 않으므로 -100의 위치를 기록해두는 label_mask를 만들어줌.

# 5. 예측

In [ ]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []

    for example in tqdm(examples):
        tokens = []
        label_mask = []
        for one_word in example:
            # 하나의 단어에 대해서 서브워드로 토큰화
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            # 서브워드 중 첫번째 서브워드를 제외하고 그 뒤의 서브워드들은 -100으로 채운다.
            label_mask.extend([0]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        # [CLS]와 [SEP]를 후에 추가할 것을 고려하여 최대 길이를 초과하는 샘플의 경우 max_seq_len - 2의 길이로 변환.
        # ex) max_seq_len = 64라면 길이가 62보다 긴 샘플은 뒷 부분을 자르고 길이 62로 변환.
        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            label_mask = label_mask[:(max_seq_len - special_tokens_count)]

        # [SEP]를 추가하는 코드
        # 1. 토큰화 결과의 맨 뒷 부분에 [SEP] 토큰 추가
        # 2. 레이블에도 맨 뒷 부분에 -100 추가.
        tokens += [sep_token]
        label_mask += [pad_token_id_for_label]

        # [CLS]를 추가하는 코드
        # 1. 토큰화 결과의 앞 부분에 [CLS] 토큰 추가
        # 2. 레이블의 맨 앞 부분에도 -100 추가.
        tokens = [cls_token] + tokens
        label_mask = [pad_token_id_for_label] + label_mask

        # 정수 인코딩
        input_id = tokenizer.convert_tokens_to_ids(tokens)

        # 어텐션 마스크 생성
        attention_mask = [1] * len(input_id)

        # 정수 인코딩에 추가할 패딩 길이 연산
        padding_count = max_seq_len - len(input_id)

        # 정수 인코딩, 어텐션 마스크에 패딩 추가
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)

        # 세그먼트 인코딩.
        token_type_id = [pad_token_id_for_segment] * max_seq_len

        # 레이블 패딩. (단, 이 경우는 패딩 토큰의 ID가 -100)
        label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_masks.append(label_mask)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    label_masks = np.asarray(label_masks, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), label_masks

In [ ]:
X_pred, label_masks = convert_examples_to_features_for_prediction(test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 5/5 [00:00<00:00, 635.44it/s]


테스트 데이터의 첫번째 샘플인 ['라티은-원윤정,', '휘닉스파크클래식', '프로골퍼']에 대해서 convert_examples_to_features_for_prediction가 변환한 결과를 확인.

In [ ]:
print('기존 원문 :', test_data_sentence[0])
print('-' * 50)
print('토큰화 후 원문 :', [tokenizer.decode([word]) for word in X_pred[0][0]])
print('레이블 마스크 :', ['[PAD]' if idx == -100 else '[FIRST]' for idx in label_masks[0]])

기존 원문 : ['라티은-원윤정,', '휘닉스파크클래식', '프로골퍼']
--------------------------------------------------
토큰화 후 원문 : ['[CLS]', '라', '##티', '##은', '-', '원', '##윤', '##정', ',', '휘', '##닉스', '##파크', '##클', '##래', '##식', '프로', '##골', '##퍼', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[P

레이블 마스크는 BERT 토크나이저가 하나의 단어에 대해서 서브워드로 분리할 경우, 첫번째 서브워드를 제외한 나머지 서브워드들에 대해서는 [PAD] 토큰. 즉, -100을 부여. 이제 모델이 임의의 문장에 대해서 예측했을 때, 레이블 마스크의 값을 참고하여 첫번째 서브워드가 아닌 뒤의 서브워드들에 대한 예측값은 무시.

In [ ]:
def ner_prediction(examples, max_seq_len, tokenizer):
  examples = [sent.split() for sent in examples]
  X_pred, label_masks = convert_examples_to_features_for_prediction(examples, max_seq_len=128, tokenizer=tokenizer)
  y_predicted = model.predict(X_pred)
  y_predicted = np.argmax(y_predicted, axis = 2)

  pred_list = []
  result_list = []

  for i in range(0, len(label_masks)):
    pred_tag = []

    # ex) 모델의 예측값 디코딩 과정
    # 예측값(y_predicted)에서 레이블 마스크(label_masks)의 값이 -100인 동일 위치의 값을 삭제
    # label_masks : [-100 0 -100 0 -100]
    # y_predicted : [  0  1   0  2   0 ] ==> [1 2] ==> 최종 예측(pred_tag) : [PER-B PER-I]
    for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
      if label_index != -100:
        pred_tag.append(index_to_tag[pred_index])

    pred_list.append(pred_tag)

  for example, pred in zip(examples, pred_list):
    one_sample_result = []
    for one_word, label_token in zip(example, pred):
      one_sample_result.append((one_word, label_token))
    result_list.append(one_sample_result)

  return result_list

훈련 데이터에 존재하지 않았던 임의의 두 개의 문장에 대해서 개체명을 예측.

In [ ]:
sent1 = '오리온스는 리그 최정상급 포인트가드 김동훈을 앞세우는 빠른 공수전환이 돋보이는 팀이다'
sent2 = '하이신사에 속한 섬들도 위로 솟아 있는데 타인은 살고 있어요'

In [ ]:
test_samples = [sent1, sent2]

In [ ]:
result_list = ner_prediction(test_samples, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 2/2 [00:00<00:00, 701.68it/s]


In [ ]:
result_list

[[('오리온스는', 'ORG-B'),
  ('리그', 'O'),
  ('최정상급', 'O'),
  ('포인트가드', 'CVL-B'),
  ('김동훈을', 'PER-B'),
  ('앞세우는', 'O'),
  ('빠른', 'O'),
  ('공수전환이', 'O'),
  ('돋보이는', 'O'),
  ('팀이다', 'O')],
 [('하이신사에', 'LOC-B'),
  ('속한', 'O'),
  ('섬들도', 'O'),
  ('위로', 'O'),
  ('솟아', 'O'),
  ('있는데', 'O'),
  ('타인은', 'O'),
  ('살고', 'O'),
  ('있어요', 'O')]]

In [ ]:
'''
다른 구현 방법!
transformers 패키지에서 제공하는 TFBertForTokenClassification를 사용해서 구현해보세요.

from transformers import TFBertForTokenClassification

model = TFBertForTokenClassification.from_pretrained("klue/bert-base", num_labels=tag_size, from_pt=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss)
'''

'\n다른 구현 방법!\ntransformers 패키지에서 제공하는 TFBertForTokenClassification를 사용해서 구현해보세요.\n\nfrom transformers import TFBertForTokenClassification\n\nmodel = TFBertForTokenClassification.from_pretrained("klue/bert-base", num_labels=tag_size, from_pt=True)\noptimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)\nmodel.compile(optimizer=optimizer, loss=model.compute_loss)\n'